In [73]:
import numpy as np
import math, json
from rdp import rdp

In [74]:
def create_animation(drawing, fps = 30, idx = 0, lw = 5): 
  
  seq_length = 0 
  
  xmax = 0 
  ymax = 0 
  
  xmin = math.inf
  ymin = math.inf
  
  #retreive min,max and the length of the drawing  
  for k in range(0, len(drawing)):
    x = drawing[k][0]
    y = drawing[k][1]

    seq_length += len(x)
    xmax = max([max(x), xmax]) 
    ymax = max([max(y), ymax]) 
    
    xmin = min([min(x), xmin]) 
    ymin = min([min(y), ymin]) 
    
  i = 0 
  j = 0
  
  # First set up the figure, the axis, and the plot element we want to animate
  fig = plt.figure()
  ax = plt.axes(xlim=(xmax+lw, xmin-lw), ylim=(ymax+lw, ymin-lw))
  ax.set_facecolor("white")
  line, = ax.plot([], [], lw=lw)

  #remove the axis 
  ax.grid = False
  ax.set_xticks([])
  ax.set_yticks([])
  
  # initialization function: plot the background of each frame
  def init():
      line.set_data([], [])
      return line, 

  # animation function.  This is called sequentially
  def animate(frame):    
    nonlocal i, j, line
    x = drawing[i][0]
    y = drawing[i][1]
    line.set_data(x[0:j], y[0:j])
    
    if j >= len(x):
      i +=1
      j = 0 
      line, = ax.plot([], [], lw=lw)
      
    else:
      j += 1
    return line,
  
  # call the animator.  blit=True means only re-draw the parts that have changed.
  anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames= seq_length + len(drawing), blit=True)
  plt.close()
  
  # save the animation as an mp4.  
  anim.save(f'video.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

In [115]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [116]:
import glob
npy_files = glob.glob('server/data/*')

In [149]:
file = np.random.choice(npy_files)
drawing = json.load(open(file))

In [161]:
drawing[0]

{'ع': [[408.29217529296875, 257.6546630859375],
  [396.12750244140625, 256.82525634765625],
  [394.3995361328125, 256.82525634765625],
  [392.87896728515625, 256.82525634765625],
  [391.427490234375, 256.9634704589844],
  [389.90692138671875, 257.3782653808594],
  [388.31719970703125, 257.79290771484375],
  [386.72747802734375, 258.3458557128906],
  [385.41424560546875, 259.03704833984375],
  [384.170166015625, 259.7282409667969],
  [383.06427001953125, 260.41943359375],
  [382.16571044921875, 261.1106262207031],
  [381.26715087890625, 261.9400939941406],
  [380.36865234375, 262.8386535644531],
  [379.53924560546875, 263.8063049316406],
  [378.91717529296875, 264.7048645019531],
  [378.36419677734375, 265.5343017578125],
  [377.94952392578125, 266.4328308105469],
  [377.88043212890625, 267.26226806640625],
  [377.88043212890625, 268.22998046875],
  [378.1568603515625, 269.2667541503906],
  [378.57159423828125, 270.3726806640625],
  [378.98626708984375, 271.5476989746094],
  [379.539245

In [156]:
post_data = []
new_data = []
total_len_strokes = 0
for item in drawing:
    xs = []
    ys = []
    char = list(item.keys())[0]
    stroke = item[char]
    if len(stroke):
        print('processing ', char)
        print('current length of stroke ', len(stroke))
        post_stroke = rdp(stroke)
        print('post length of stroke ', len(post_stroke))
        total_len_strokes += len(post_stroke)
        for x, y in post_stroke:
            xs.append(500 - x)
            ys.append(y)
        post_data.append([xs, ys])
        xs = []
        ys = []
        for x, y in stroke:
            xs.append(500 - x)
            ys.append(y)
        new_data.append([xs, ys])

print(total_len_strokes)

processing  ع
current length of stroke  98
post length of stroke  95
processing  ٮ
current length of stroke  57
post length of stroke  54
processing  .
current length of stroke  1
post length of stroke  1
processing  د
current length of stroke  78
post length of stroke  76
processing  ا
current length of stroke  55
post length of stroke  45
processing  ل
current length of stroke  56
post length of stroke  47
processing  ر
current length of stroke  70
post length of stroke  68
processing  س
current length of stroke  70
post length of stroke  63
processing  و
current length of stroke  76
post length of stroke  71
processing  ل
current length of stroke  94
post length of stroke  91
611


In [160]:
len(post_data)

10

In [151]:
create_animation(data)

In [152]:
import io
import base64
video = io.open('video.mp4', 'r+b').read()

encoded = base64.b64encode(video)
HTML(data='''<video alt="video" autoplay>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))